In [6]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pickle
import glob
from tqdm.notebook import trange, tqdm
import json
import re
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
from varying_doc_sizes import SIZES
from train_utils import generate_doc_pairs_no_chunking
from collections import defaultdict

import tikzplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib notebook

In [8]:

from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [13]:
FIGURES_PATH = '../figures/'
COMPUTED_DATA_PATH = '../temp_data/reddit/preprocessed/'
EXPERIMENT_DATA_PATH = '../temp_data/reddit_old/doc_size_experiments/'

Load HPC Data
===

Dataset Stats
---

In [10]:
with open(COMPUTED_DATA_PATH + 'metadata.p', 'rb') as f:
    (train_files, test_files, min_count, author_mapping) = pickle.load(f)

In [14]:
with open(EXPERIMENT_DATA_PATH + 'size_' + str(SIZES[0]) + '/training_meta_data.p', 'rb') as f:
    (author_to_doc_idx, author_to_doc_idx_test, author_subreddit, author_subreddit_test, x_shape, x_shape_test) = pickle.load(f)

In [15]:
author_mapping = defaultdict(set)
author_to_root = {}
for y in author_to_doc_idx.keys():
    u = re.search(r'(.*)_[A-Z]+$', y).group(1)
    author_mapping[u].add(y)
    author_to_root[y] = u

subreddit_to_author = defaultdict(list)
for k, v in author_subreddit.items():
    subreddit_to_author[v].append(k)
X_idxs_train, Y_train = generate_doc_pairs_no_chunking(author_mapping, subreddit_to_author, author_to_root, author_to_doc_idx, author_subreddit)

new2



20529 37405 32646
20529 6158 14370


In [16]:
print('Num train users:', len(author_mapping))
print('Num train user pairs:', len(Y_train))

Num train users: 23224
Num train user pairs: 41057


AUCS
--

In [12]:
aucs = []
for size in SIZES:
    with open(EXPERIMENT_DATA_PATH + 'size_' + str(size) + '/model.p', 'rb') as f:
        data = pickle.load(f)
        probs = data[5]
        Y_test = data[6]
        fpr, tpr, thresh = roc_curve(Y_test, probs)
        roc_auc = auc(fpr, tpr)
        print(size, roc_auc)
        aucs.append(roc_auc)

FileNotFoundError: [Errno 2] No such file or directory: '../temp_data/reddit/doc_size_experiments/size_5/model.p'

In [ ]:
style.use('seaborn-paper')

fig, ax = plt.subplots()
plt_x = SIZES
plt_y = aucs
plt.plot(plt_x, plt_y, '-o')

plt.xlabel('Comments')
plt.ylabel('AUC')
plt.grid(True, alpha=0.5)
# plt.tight_layout()
plt.ylim(0.6, 1)
plt.show()

tikzplotlib.save(FIGURES_PATH + 'plt_reddit_doc_size_vs_auc.pgf')